<a href="https://colab.research.google.com/github/indrawanal/PREDIIKSI-LELE-TEST-ALTAP/blob/main/new_lstm_test_lele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries yang diperlukan
!pip install pandas numpy matplotlib seaborn tensorflow scikit-learn

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snis
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# URL dataset
url = 'https://github.com/AryaPratamaPutra-10/Analys_Data_Lele/raw/75a3f74df5b05158dbf856cca04b977a20d67a47/Data_Lele_1.csv'

# Membaca dataset
df = pd.read_csv(url, sep=';')
print(df.head(100))  # Menampilkan 100 baris pertama



    Id    Provinsi Kabupaten/Kota Jenis Ikan  Tahun  Volume Produksi  \
0  NaN  JAWA BARAT        BANDUNG       LELE   2019          5372785   
1  NaN  JAWA BARAT  BANDUNG BARAT       LELE   2019          1328082   
2  NaN  JAWA BARAT         BEKASI       LELE   2019          2019121   
3  NaN  JAWA BARAT          BOGOR       LELE   2019         89814340   
4  NaN  JAWA BARAT         CIAMIS       LELE   2019         18162827   
..  ..         ...            ...        ...    ...              ...   
95 NaN  JAWA TIMUR      SITUBONDO       LELE   2019           185865   
96 NaN  JAWA TIMUR        SUMENEP       LELE   2019           170428   
97 NaN  JAWA TIMUR     TRENGGALEK       LELE   2019          4053426   
98 NaN  JAWA TIMUR          TUBAN       LELE   2019          2010670   
99 NaN  JAWA TIMUR    TULUNGAGUNG       LELE   2019         10006869   

    Nilai Produksi   Harga Lele  
0      85964560000  16000.00000  
1      19921230000  15000.00000  
2      34325057000  17000.00000  

In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# URL dataset
url = 'https://github.com/AryaPratamaPutra-10/Analys_Data_Lele/raw/75a3f74df5b05158dbf856cca04b977a20d67a47/Data_Lele_1.csv'

# Membaca dataset
df = pd.read_csv(url, sep=';')
# Memastikan tipe data yang benar
df['Tahun'] = df['Tahun'].astype(str)

# One-Hot Encoding untuk kolom 'Kabupaten/Kota' dan 'Provinsi'
df_encoded = pd.get_dummies(df, columns=['Kabupaten/Kota', 'Provinsi'])

# Fitur yang akan digunakan
features = ['Volume Produksi', 'Nilai Produksi', 'Harga Lele'] + [col for col in df_encoded.columns if 'Kabupaten/Kota' in col or 'Provinsi' in col]



# Menyiapkan data input dan target
X = df_encoded[features].values
y = df_encoded['Harga Lele'].values  # Target: Harga Lele

# Normalisasi data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Mengubah data menjadi bentuk yang sesuai untuk LSTM (samples, time_steps, features)
# Misalnya, kita menggunakan 1 waktu langkah (time step) untuk prediksi harga ke depan
X_train_3d = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_3d = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Membuat model LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Latih model
history = model.fit(X_train_3d, y_train, epochs=50, batch_size=32, validation_data=(X_test_3d, y_test))

# Evaluasi model
loss = model.evaluate(X_test_3d, y_test)
print(f'Loss pada data uji: {loss}')

# Prediksi harga lele ke depan
predictions = model.predict(X_test_3d)
print(f'Prediksi harga lele: {predictions[:50]}')  # Menampilkan 50 prediksi pertama


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0790 - val_loss: 0.0536
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0513 - val_loss: 0.0324
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0284 - val_loss: 0.0195
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0153 - val_loss: 0.0148
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0104 - val_loss: 0.0133
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0077 - val_loss: 0.0111
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0077 - val_loss: 0.0104
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0056 - val_loss: 0.0098
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0054 - val_loss: 0.0093
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0046 - val_loss: 0.0089
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0048 - val_loss

In [46]:
# Mengembalikan prediksi ke skala asli
predicted_prices = scaler.inverse_transform(predictions)
print(predicted_prices)


[[13072.364 ]
 [15975.029 ]
 [16897.867 ]
 [17526.836 ]
 [12805.465 ]
 [16017.59  ]
 [14968.663 ]
 [17678.37  ]
 [14974.571 ]
 [17021.408 ]
 [11658.134 ]
 [15993.525 ]
 [19097.625 ]
 [15325.554 ]
 [17547.87  ]
 [17248.193 ]
 [17971.982 ]
 [17246.812 ]
 [28179.79  ]
 [22209.324 ]
 [18033.605 ]
 [19913.242 ]
 [15475.812 ]
 [18141.668 ]
 [17063.717 ]
 [13787.027 ]
 [17268.65  ]
 [14999.337 ]
 [15455.393 ]
 [16213.046 ]
 [18386.117 ]
 [19965.572 ]
 [16913.8   ]
 [15924.849 ]
 [17746.615 ]
 [18669.752 ]
 [13379.394 ]
 [13838.601 ]
 [11658.15  ]
 [15917.569 ]
 [15966.522 ]
 [18584.857 ]
 [17474.51  ]
 [16339.059 ]
 [16522.773 ]
 [15122.116 ]
 [16086.798 ]
 [14998.932 ]
 [16298.114 ]
 [16183.395 ]
 [16411.447 ]
 [15749.071 ]
 [16450.05  ]
 [19502.062 ]
 [18858.885 ]
 [17539.727 ]
 [16552.25  ]
 [11900.959 ]
 [12808.08  ]
 [15851.101 ]
 [16053.097 ]
 [17976.623 ]
 [19410.787 ]
 [17145.512 ]
 [18738.025 ]
 [16876.59  ]
 [18680.062 ]
 [17298.012 ]
 [16635.695 ]
 [18206.46  ]
 [17422.193 ]
 [1701

In [47]:
import pandas as pd
import numpy as np

# Misalnya, Anda memiliki tanggal mulai
start_date = '2024-01-01'  # tanggal awal untuk prediksi

# Membuat rentang tanggal berdasarkan jumlah prediksi
date_range = pd.date_range(start=start_date, periods=len(predicted_prices), freq='D')  # D untuk harian, Anda bisa sesuaikan frekuensinya

# Membuat DataFrame dengan prediksi harga dan tanggal
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Prediksi Harga Lele'])
predicted_prices_df['Tanggal'] = date_range  # Menambahkan kolom tanggal

# Menambahkan kolom tahun, bulan, dan hari jika diperlukan
predicted_prices_df['Tahun'] = predicted_prices_df['Tanggal'].dt.year
predicted_prices_df['Bulan'] = predicted_prices_df['Tanggal'].dt.month
predicted_prices_df['Hari'] = predicted_prices_df['Tanggal'].dt.day

# Menampilkan 10 baris pertama prediksi harga dengan tanggal dan tahun
print(predicted_prices_df.head(20))


    Prediksi Harga Lele    Tanggal  Tahun  Bulan  Hari
0          13072.364258 2024-01-01   2024      1     1
1          15975.029297 2024-01-02   2024      1     2
2          16897.867188 2024-01-03   2024      1     3
3          17526.835938 2024-01-04   2024      1     4
4          12805.464844 2024-01-05   2024      1     5
5          16017.589844 2024-01-06   2024      1     6
6          14968.663086 2024-01-07   2024      1     7
7          17678.369141 2024-01-08   2024      1     8
8          14974.571289 2024-01-09   2024      1     9
9          17021.408203 2024-01-10   2024      1    10
10         11658.133789 2024-01-11   2024      1    11
11         15993.525391 2024-01-12   2024      1    12
12         19097.625000 2024-01-13   2024      1    13
13         15325.553711 2024-01-14   2024      1    14
14         17547.869141 2024-01-15   2024      1    15
15         17248.193359 2024-01-16   2024      1    16
16         17971.982422 2024-01-17   2024      1    17
17        

In [34]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Prediksi model
predictions = model.predict(x_test)  # Gantilah x_test dengan data uji yang sesuai
predictions = scaler.inverse_transform(predictions)  # Kembalikan ke skala asli

# Menghitung Metrik Evaluasi
mae = mean_absolute_error(y_test, predictions)  # Gantilah y_test dengan nilai asli target
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

# Menampilkan Hasil Evaluasi
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')


NameError: name 'x_test' is not defined